In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

driver = webdriver.Chrome()

# Entering website
url = 'https://www.linkedin.com/'
driver.get(url)
time.sleep(5)

# Function to check for the login page and go back
def check_and_go_back_if_login_page():
    try:
        login_element = driver.find_element(By.XPATH, '//*[@id="join-form-submit"]')
        if login_element:
            print("Login page detected. Going back to continue scraping.")
            driver.back()
            time.sleep(3)  # Wait for the previous page to load
            return True
    except:
        return False
    return False

def scrape_jobs():
    driver.find_element(By.XPATH, '/html/body/nav/ul/li[4]').click()
    time.sleep(3)
    search_bar = driver.find_element(By.XPATH, '//*[@id="job-search-bar-keywords"]')
    search_bar.click()
    
    search_query = "Data Analyst"
    search_bar.send_keys(search_query)
    
    # Press Enter to search
    search_bar.send_keys(Keys.ENTER)
    time.sleep(3)
    
    driver.find_element(By.XPATH, '//*[@id="jobs-search-panel"]/form/section[2]/button').click()
    time.sleep(3)
    search_bar = driver.find_element(By.XPATH, '//*[@id="job-search-bar-location"]')
    search_bar.click()

    # Clear the location and enter a new one
    search_bar.send_keys(Keys.CONTROL + "a")
    search_bar.send_keys(Keys.DELETE)
    search_query = "Dhaka"
    search_bar.send_keys(search_query)
    time.sleep(3)
    
    # Press Enter to search
    search_bar.send_keys(Keys.ENTER)
    time.sleep(3)

    #selecting filter option
    driver.find_element(By.XPATH, '//*[@id="jserp-filters"]/ul/li[2]/div/div/button').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="f_TPR-1"]').click()
    driver.find_element(By.XPATH, '//*[@id="jserp-filters"]/ul/li[2]/div/div/div/button').click()
    time.sleep(3)

    # Extract the total number of jobs
    total_jobs_element = driver.find_element(By.XPATH, '//*[@id="main-content"]/div/h1/span[1]')
    total_jobs_text = total_jobs_element.text.split()[0].replace(',', '')
    total_jobs = int(total_jobs_text)

    print(f"Total jobs found: {total_jobs}")
    
    Title = []
    Company = []
    Link = []
    Time = []
    Level = []
    Type = []
    Funtion = []
    Industry = []
    
    for i in range(1, 6):
        try:
            if check_and_go_back_if_login_page():
                continue
                
            job_listing_xpath = f'//*[@id="main-content"]/section[2]/ul/li[{i}]/div/a'
            job_link = driver.find_element(By.XPATH, job_listing_xpath)
            
            link = job_link.get_attribute('href')
            
            job_link.click()
            time.sleep(6)

            if check_and_go_back_if_login_page():
                # Re-click the job link after going back
                job_link = driver.find_element(By.XPATH, job_listing_xpath)
                job_link.click()
                time.sleep(3)

            # Extract job details
            job_title = driver.find_element(By.XPATH, '//*[@id="main-content"]//h1').text
            job_company = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[1]/span[1]/a').text
            times = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/section[2]/div/div[1]/div/h4/div[2]/span').text
            level = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[1]/span').text
            type = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[2]/span').text
            func = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[3]/span').text
            industry = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/ul/li[4]/span').text

            # Append data to lists only if all elements are successfully retrieved
            Title.append(job_title)
            Company.append(job_company)
            Link.append(link)        
            Time.append(times)
            Level.append(level)
            Type.append(type)
            Funtion.append(func)
            Industry.append(industry)
            
            driver.back()
            time.sleep(3)
            
        except Exception as e:
            print(f"Error scraping job {i}: {e}")


    # Convert the data to a DataFrame
    df = pd.DataFrame({
        'Title': Title,
        'Company': Company,
        'Link': Link,
        'Time': Time,
        'Level': Level,
        'Type': Type,
        'Funtion': Funtion,
        'Industry': Industry,
    })
    
    print(df)
    df.to_csv('C:/Users/Saiful Hasan/Downloads/job_list.csv', index=False)

scrape_jobs()

driver.quit()


Total jobs found: 319
                             Title               Company  \
0                 Report Developer  Kontoor Brands, Inc.   
1  AI Researcher (Computer Vision)       Datalytics Labs   
2               IDT Data Scientist                   BAT   
3                  System Engineer                 Wipro   

                                                Link         Time  \
0  https://bd.linkedin.com/jobs/view/report-devel...   1 week ago   
1  https://bd.linkedin.com/jobs/view/ai-researche...  8 hours ago   
2  https://bd.linkedin.com/jobs/view/idt-data-sci...   1 week ago   
3  https://bd.linkedin.com/jobs/view/system-engin...   2 days ago   

              Level       Type                                 Funtion  \
0  Mid-Senior level  Full-time                  Information Technology   
1        Internship  Full-time  Engineering and Information Technology   
2  Mid-Senior level  Full-time                    Research and Science   
3    Not Applicable  Full-time     